In [25]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [26]:
# %cd "/content/drive/MyDrive/DeepLearning/CS7643 Deep Learning Project/RelTR"
%cd "/content/drive/MyDrive/CS7643 Deep Learning Project/RelTR"

/content/drive/.shortcut-targets-by-id/1qHoaQqsa_84wKp4XFuiBiBUnxjcdMWJm/CS7643 Deep Learning Project/RelTR


In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as T
import numpy as np
import random
import time

from PIL import Image
import requests
import matplotlib.pyplot as plt

# Initialize RelTr Backnbone

In [29]:
from models.backbone import Backbone, Joiner
from models.position_encoding import PositionEmbeddingSine
from models.transformer import Transformer
from models.reltr import RelTR

position_embedding = PositionEmbeddingSine(128, normalize=True)
backbone = Backbone('resnet50', False, False, False)
backbone = Joiner(backbone, position_embedding)
backbone.num_channels = 2048

transformer = Transformer(d_model=256, dropout=0.1, nhead=8,
                          dim_feedforward=2048,
                          num_encoder_layers=6,
                          num_decoder_layers=6,
                          normalize_before=False,
                          return_intermediate_dec=True)

reltr_model = RelTR(backbone, transformer, num_classes=151, num_rel_classes = 51,
              num_entities=100, num_triplets=200)

# The checkpoint is pretrained on Visual Genome
device = "cuda" if torch.cuda.is_available() else "cpu"
ckpt = torch.load('ckpt/checkpoint0149.pth', map_location=torch.device(device), weights_only=False)
reltr_model.load_state_dict(ckpt['model'])
reltr_model.eval()

RelTR(
  (transformer): Transformer(
    (encoder): TransformerEncoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerEncoderLayer(
          (self_attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
          )
          (linear1): Linear(in_features=256, out_features=2048, bias=True)
          (dropout): Dropout(p=0.1, inplace=False)
          (linear2): Linear(in_features=2048, out_features=256, bias=True)
          (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
          (dropout1): Dropout(p=0.1, inplace=False)
          (dropout2): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (decoder): TransformerDecoder(
      (layers): ModuleList(
        (0-5): 6 x TransformerDecoderLayer(
          (self_attn_entity): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features

# Freeze params

In [30]:
for param in reltr_model.parameters():
    param.requires_grad = False

# Initialize arguments for model loading and training

In [31]:
# from types import SimpleNamespace

# device = "cuda" if torch.cuda.is_available() else "cpu"

# args = {
#     'bbox_loss_coef': 5,
#     'giou_loss_coef': 2,
#     'rel_loss_coef': 1,
#     'aux_loss': False,
#     'set_cost_class': 1,
#     'set_cost_bbox': 5,
#     'set_cost_giou': 2,
#     'set_iou_threshold':0.7,
#     'dataset': 'vg',
#     'device': device,
#     'eos_coef': 0.1,
#     'seed': 42,
#     'lr_backbone': 1e-5,
#     'lr': 1e-4,
#     'lr_drop': 200,
#     'weight_decay': 1e-4,
#     'ann_path': 'data/vg/',
#     'img_folder': 'data/vg/images',
#     'eval': False,
#     'batch_size': 10,
#     'epochs': 10,
#     'num_workers': 2,
# }
# args = SimpleNamespace(**args)

# Build Model and Load it

In [32]:
# from deecap import DeecapRelTR
# from models.backbone import build_backbone
# from models.matcher import build_matcher
# from models.transformer import build_transformer
# from models.reltr import SetCriterion, PostProcess
# from datasets import build_dataset
# import util.misc as utils
# from torch.utils.data import DataLoader, DistributedSampler


# def build_model(args):
#   num_classes = 151 if args.dataset != 'oi' else 289 # some entity categories in OIV6 are deactivated.
#   num_rel_classes = 51 if args.dataset != 'oi' else 31

#   device = torch.device(args.device)

#   matcher = build_matcher(args)

#   model = DeecapRelTR(reltr_model)

#   model.to(device)

#   weight_dict = {'loss_ce': 1, 'loss_bbox': args.bbox_loss_coef}
#   weight_dict['loss_giou'] = args.giou_loss_coef
#   weight_dict['loss_rel'] = args.rel_loss_coef

#   # TODO this is a hack
#   if args.aux_loss:
#       aux_weight_dict = {}
#       for i in range(args.dec_layers - 1):
#           aux_weight_dict.update({k + f'_{i}': v for k, v in weight_dict.items()})
#       weight_dict.update(aux_weight_dict)

#   losses = ['labels', 'boxes', 'cardinality', "relations"]

#   criterion = SetCriterion(num_classes, num_rel_classes, matcher=matcher, weight_dict=weight_dict,
#                             eos_coef=args.eos_coef, losses=losses)
#   criterion.to(device)
#   postprocessors = {'bbox': PostProcess()}
#   return model, criterion, postprocessors

In [33]:
# utils.init_distributed_mode(args)

# seed = args.seed + utils.get_rank()
# torch.manual_seed(seed)
# np.random.seed(seed)
# random.seed(seed)

# model, criterion, postprocessors = build_model(args)
# model.to(device)

# optimizer = torch.optim.Adam([
#     {'params': model.imitation_networks_enitity.parameters(), 'lr': 1e-4},
#     {'params': model.imitation_networks_subject.parameters(), 'lr': 1e-4},
#     {'params': model.imitation_networks_object.parameters(), 'lr': 1e-4},
#     {'params': model.imitation_networks_submap.parameters(), 'lr': 1e-4},
#     {'params': model.imitation_networks_objmap.parameters(), 'lr': 1e-4},
# ])

# lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, args.lr_drop)

# dataset_train = build_dataset(image_set='train', args=args) #need to add dataset
# dataset_val = build_dataset(image_set='val', args=args)


# if args.distributed:
#         sampler_train = DistributedSampler(dataset_train)
#         sampler_val = DistributedSampler(dataset_val, shuffle=False)
# else:
#     sampler_train = torch.utils.data.RandomSampler(dataset_train)
#     sampler_val = torch.utils.data.SequentialSampler(dataset_val)


# batch_sampler_train = torch.utils.data.BatchSampler(
#     sampler_train, args.batch_size, drop_last=True)

# data_loader_train = DataLoader(dataset_train, batch_sampler=batch_sampler_train,
#                                 collate_fn=utils.collate_fn, num_workers=args.num_workers)
# data_loader_val = DataLoader(dataset_val, args.batch_size, sampler=sampler_val,
#                               drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)


# Load pre-trained weights

- model_weights_10.pth - model trained with no decoder layer mixing for 10 epochs
- model_weights_average_mixing.pth -  model trained with decoder layer mixing (average) for 10 epochs

In [34]:
# model.load_state_dict(torch.load('model_weights_avg_mix_10.pth', map_location=torch.device('cuda')))
# model.to(device)

# Training Loop (For Reference)

In [35]:
# # Lambda for balancing imitation loss
# lambda_imitation = 0.5
# epochs = 10

# # Set the model to training mode
# model.train()

# for epoch in range(epochs):
#     running_loss = 0.0  # To accumulate the loss for each epoch
#     for batch_idx, (images, targets) in enumerate(data_loader_train):

#         images = images.to(device)
#         targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

#         optimizer.zero_grad()

#         predictions = model(images)
#         # print(predictions["pred_logits"].shape[:2])

#         imitation_loss, task_losses = model.compute_imitation_loss(criterion, targets)

#         total_loss = lambda_imitation * imitation_loss + (1 - lambda_imitation) * task_losses

#         # Backward pass (calculate gradients)
#         total_loss.backward()

#         # Update the model parameters
#         optimizer.step()

#         # Accumulate the running loss for logging
#         running_loss += total_loss.item()

#         if (batch_idx + 1) % 10 == 0:
#             avg_loss = running_loss / (batch_idx + 1)  # Calculate average loss
#             print(f"Epoch [{epoch+1}/{epochs}], Batch [{batch_idx+1}/{len(data_loader_train)}], "
#                   f"Avg Loss: {avg_loss:.4f}, Imitation Loss: {imitation_loss.item():.4f}, Task Loss: {task_losses.item():.4f}")

#     avg_epoch_loss = running_loss / len(data_loader_train)
#     print(f"End of Epoch [{epoch+1}/{epochs}], Average Total Loss: {avg_epoch_loss:.4f}")

In [36]:
torch.cuda.empty_cache()
torch.cuda.ipc_collect()

# Metrics [TODO]

In [37]:
from lib.evaluation.sg_eval import BasicSceneGraphEvaluator
import torch
import torch.nn.functional as F
from torchvision.ops import box_iou
from collections import defaultdict
import numpy as np
CLASSES = [ 'N/A', 'airplane', 'animal', 'arm', 'bag', 'banana', 'basket', 'beach', 'bear', 'bed', 'bench', 'bike',
                'bird', 'board', 'boat', 'book', 'boot', 'bottle', 'bowl', 'box', 'boy', 'branch', 'building',
                'bus', 'cabinet', 'cap', 'car', 'cat', 'chair', 'child', 'clock', 'coat', 'counter', 'cow', 'cup',
                'curtain', 'desk', 'dog', 'door', 'drawer', 'ear', 'elephant', 'engine', 'eye', 'face', 'fence',
                'finger', 'flag', 'flower', 'food', 'fork', 'fruit', 'giraffe', 'girl', 'glass', 'glove', 'guy',
                'hair', 'hand', 'handle', 'hat', 'head', 'helmet', 'hill', 'horse', 'house', 'jacket', 'jean',
                'kid', 'kite', 'lady', 'lamp', 'laptop', 'leaf', 'leg', 'letter', 'light', 'logo', 'man', 'men',
                'motorcycle', 'mountain', 'mouth', 'neck', 'nose', 'number', 'orange', 'pant', 'paper', 'paw',
                'people', 'person', 'phone', 'pillow', 'pizza', 'plane', 'plant', 'plate', 'player', 'pole', 'post',
                'pot', 'racket', 'railing', 'rock', 'roof', 'room', 'screen', 'seat', 'sheep', 'shelf', 'shirt',
                'shoe', 'short', 'sidewalk', 'sign', 'sink', 'skateboard', 'ski', 'skier', 'sneaker', 'snow',
                'sock', 'stand', 'street', 'surfboard', 'table', 'tail', 'tie', 'tile', 'tire', 'toilet', 'towel',
                'tower', 'track', 'train', 'tree', 'truck', 'trunk', 'umbrella', 'vase', 'vegetable', 'vehicle',
                'wave', 'wheel', 'window', 'windshield', 'wing', 'wire', 'woman', 'zebra']

REL_CLASSES = ['__background__', 'above', 'across', 'against', 'along', 'and', 'at', 'attached to', 'behind',
                'belonging to', 'between', 'carrying', 'covered in', 'covering', 'eating', 'flying in', 'for',
                'from', 'growing on', 'hanging from', 'has', 'holding', 'in', 'in front of', 'laying on',
                'looking at', 'lying on', 'made of', 'mounted on', 'near', 'of', 'on', 'on back of', 'over',
                'painted on', 'parked on', 'part of', 'playing', 'riding', 'says', 'sitting on', 'standing on',
                'to', 'under', 'using', 'walking in', 'walking on', 'watching', 'wearing', 'wears', 'with']

In [38]:
# evaluator = BasicSceneGraphEvaluator(mode='sgdet', multiple_preds=True)

In [39]:
# CONF_THRESH = 0.3
# CLS_THRESH = 0.3
# RECALL_K = [20, 50, 100]
# MATCH_SCORE_THRESH = 0.2

# def box_cxcywh_to_xyxy(x):
#     x_c, y_c, w, h = x.unbind(-1)
#     return torch.stack((x_c - 0.5 * w, y_c - 0.5 * h,
#                         x_c + 0.5 * w, y_c + 0.5 * h), dim=-1)

# def soft_match_and_index(pred_sub_box, pred_obj_box, pred_r, pred_sub_class, pred_obj_class,
#                          gt_sub_boxes, gt_obj_boxes, gt_pred_ids, gt_sub_classes, gt_obj_classes):
#     iou_sub = box_iou(pred_sub_box.unsqueeze(0), gt_sub_boxes).squeeze(0)
#     iou_obj = box_iou(pred_obj_box.unsqueeze(0), gt_obj_boxes).squeeze(0)

#     best_score = 0
#     best_idx = -1
#     for i in range(len(gt_pred_ids)):
#         score = 0.4 * iou_sub[i].item() + 0.4 * iou_obj[i].item()
#         if pred_r == gt_pred_ids[i].item():
#             score += 0.1
#         if pred_sub_class == gt_sub_classes[i].item():
#             score += 0.05
#         if pred_obj_class == gt_obj_classes[i].item():
#             score += 0.05
#         if score > best_score:
#             best_score = score
#             best_idx = i
#     return best_score, best_idx

# def align_predictions_to_boxes(mode, sub_classes, obj_classes, pred_rels,
#                                 sub_boxes, obj_boxes, max_k=100):
#     """
#     Ensures all prediction tensors are trimmed to the same safe length.
#     Returns consistently aligned tensors.
#     """
#     max_len = min(len(pred_rels), len(sub_classes), len(obj_classes),
#                   len(sub_boxes), len(obj_boxes), max_k)
#     return (
#         sub_classes[:max_len],
#         obj_classes[:max_len],
#         pred_rels[:max_len],
#         sub_boxes[:max_len],
#         obj_boxes[:max_len]
#     )


# def evaluate_triplets(model, data_loader_val, device, mode='predcls'):
#     assert mode in ['predcls', 'sgcls', 'sgdet']
#     model.eval()

#     total_gt_rels = 0
#     recall_counts = {k: 0 for k in RECALL_K}
#     all_recalls = defaultdict(list)

#     for batch_idx, (samples, targets) in enumerate(data_loader_val):
#         print(f"Evaluating batch {batch_idx + 1}/{len(data_loader_val)}")
#         samples = samples.to(device)
#         with torch.no_grad():
#             outputs = model(samples, layer_exit=2)

#         for i in range(len(targets)):
#             gt = targets[i]
#             if gt['rel_annotations'].shape[0] == 0:
#                 continue

#             rel_probs = F.softmax(outputs['rel_logits'][i][:, :-1], dim=-1)
#             sub_probs = F.softmax(outputs['sub_logits'][i][:, :-1], dim=-1)
#             obj_probs = F.softmax(outputs['obj_logits'][i][:, :-1], dim=-1)

#             if mode in ['predcls', 'sgdet']:
#                 keep = (
#                     (rel_probs.max(-1).values > CONF_THRESH) &
#                     (sub_probs.max(-1).values > CONF_THRESH) &
#                     (obj_probs.max(-1).values > CONF_THRESH)
#                 ).cpu()
#             else:  # sgcls
#                 keep = torch.ones(rel_probs.shape[0], dtype=torch.bool)

#             if keep.sum() == 0:
#                 continue

#             rel_probs = rel_probs[keep]
#             sub_probs = sub_probs[keep]
#             obj_probs = obj_probs[keep]

#             scores = (
#                 rel_probs.max(-1).values *
#                 sub_probs.max(-1).values *
#                 obj_probs.max(-1).values
#             )
#             sorted_idx = scores.argsort(descending=True).cpu()

#             rel_probs = rel_probs[sorted_idx]
#             sub_probs = sub_probs[sorted_idx]
#             obj_probs = obj_probs[sorted_idx]

#             sub_classes = sub_probs.argmax(dim=1).cpu()
#             obj_classes = obj_probs.argmax(dim=1).cpu()
#             pred_rels = rel_probs.argmax(dim=1).cpu()

#             # boxes
#             if mode == 'sgdet':
#                 img_h, img_w = targets[i]['orig_size'].tolist()
#                 scale_fct = torch.tensor([img_w, img_h, img_w, img_h], device=device)

#                 sub_boxes = (
#                     box_cxcywh_to_xyxy(outputs['sub_boxes'][i][keep][sorted_idx]) * scale_fct
#                 ).cpu()
#                 obj_boxes = (
#                     box_cxcywh_to_xyxy(outputs['obj_boxes'][i][keep][sorted_idx]) * scale_fct
#                 ).cpu()
#             else:
#                 gt_boxes = gt['boxes'].cpu()
#                 rel_anno = gt['rel_annotations']
#                 sub_ids = rel_anno[:, 0]
#                 obj_ids = rel_anno[:, 1]
#                 sub_boxes = box_cxcywh_to_xyxy(gt_boxes[sub_ids])
#                 obj_boxes = box_cxcywh_to_xyxy(gt_boxes[obj_ids])

#             sub_classes, obj_classes, pred_rels, sub_boxes, obj_boxes = align_predictions_to_boxes(
#                 mode, sub_classes, obj_classes, pred_rels, sub_boxes, obj_boxes, max_k=100
#             )

#             # GT triplets
#             gt_boxes = gt['boxes'].cpu()
#             gt_classes = gt['labels'].cpu()
#             gt_rels = gt['rel_annotations'].cpu()
#             gt_sub_ids = gt_rels[:, 0]
#             gt_obj_ids = gt_rels[:, 1]
#             gt_pred_ids = gt_rels[:, 2]
#             total_gt_rels += len(gt_pred_ids)

#             gt_sub_boxes = box_cxcywh_to_xyxy(gt_boxes[gt_sub_ids])
#             gt_obj_boxes = box_cxcywh_to_xyxy(gt_boxes[gt_obj_ids])
#             gt_sub_classes = gt_classes[gt_sub_ids]
#             gt_obj_classes = gt_classes[gt_obj_ids]

#             for k in RECALL_K:
#                 hits = 0
#                 matched_gt = set()
#                 for j in range(min(k, len(pred_rels))):
#                     score, match_idx = soft_match_and_index(
#                         sub_boxes[j], obj_boxes[j], pred_rels[j],
#                         sub_classes[j], obj_classes[j],
#                         gt_sub_boxes, gt_obj_boxes,
#                         gt_pred_ids, gt_sub_classes, gt_obj_classes
#                     )
#                     if score >= MATCH_SCORE_THRESH and match_idx not in matched_gt:
#                         hits += 1
#                         matched_gt.add(match_idx)
#                 recall_counts[k] += hits
#                 all_recalls[k].append(hits / len(gt_pred_ids))

#         break
#     print(f"\n==== FINAL {mode.upper()} EVALUATION (SOFT MATCH) ====")
#     num_images = sum(len(v) for v in all_recalls.values()) // len(RECALL_K)
#     for k in RECALL_K:
#         recall = recall_counts[k] / total_gt_rels
#         mean_recall = np.mean(all_recalls[k])
#         precision = recall_counts[k] / (num_images * k)
#         f1 = 2 * precision * recall / (precision + recall + 1e-8)
#         print(f"Recall@{k:3d}: {recall:.3f}   mRecall@{k:3d}: {mean_recall:.3f}   "
#               f"Precision@{k:3d}: {precision:.3f}   F1@{k:3d}: {f1:.3f}")


In [40]:
# evaluate_triplets(model, data_loader_val, device, mode='predcls')
# evaluate_triplets(model, data_loader_val, device, mode='sgcls')
# evaluate_triplets(model, data_loader_val, device, mode='sgdet')

In [41]:
from types import SimpleNamespace

device = "cuda" if torch.cuda.is_available() else "cpu"

args = {
    'bbox_loss_coef': 5,
    'giou_loss_coef': 2,
    'rel_loss_coef': 1,
    'aux_loss': False,
    'set_cost_class': 1,
    'set_cost_bbox': 5,
    'set_cost_giou': 2,
    'set_iou_threshold':0.7,
    'dataset': 'vg',
    'device': device,
    'eos_coef': 0.1,
    'seed': 42,
    'lr_backbone': 1e-5,
    'lr': 1e-4,
    'lr_drop': 200,
    'weight_decay': 1e-4,
    'ann_path': 'data/vg/',
    'img_folder': 'data/vg/images',
    'eval': True,
    'batch_size': 10,
    'epochs': 10,
    'num_workers': 2,
    'exit': 3
}
args = SimpleNamespace(**args)

In [42]:
from deecap import DeecapRelTR
from models.backbone import build_backbone
from models.matcher import build_matcher
from models.transformer import build_transformer
from models.reltr import SetCriterion, PostProcess
from datasets import build_dataset
import util.misc as utils
from torch.utils.data import DataLoader, DistributedSampler

def build_model_for_eval(args):
  num_classes = 151 if args.dataset != 'oi' else 289 # some entity categories in OIV6 are deactivated.
  num_rel_classes = 51 if args.dataset != 'oi' else 31

  device = torch.device(args.device)

  matcher = build_matcher(args)

  model = DeecapRelTR(reltr_model)
  model.load_state_dict(torch.load('./model weights/avg_mix_50_0.7.pth', map_location=torch.device('cuda')))

  model.to(device)

  weight_dict = {'loss_ce': 1, 'loss_bbox': args.bbox_loss_coef}
  weight_dict['loss_giou'] = args.giou_loss_coef
  weight_dict['loss_rel'] = args.rel_loss_coef

  # TODO this is a hack
  if args.aux_loss:
      aux_weight_dict = {}
      for i in range(args.dec_layers - 1):
          aux_weight_dict.update({k + f'_{i}': v for k, v in weight_dict.items()})
      weight_dict.update(aux_weight_dict)

  losses = ['labels', 'boxes', 'cardinality', "relations"]

  criterion = SetCriterion(num_classes, num_rel_classes, matcher=matcher, weight_dict=weight_dict,
                            eos_coef=args.eos_coef, losses=losses)
  criterion.to(device)
  postprocessors = {'bbox': PostProcess()}
  return model, criterion, postprocessors

In [49]:
import datasets
import util.misc as utils
from datasets import build_dataset, get_coco_api_from_dataset
from engine import evaluate, train_one_epoch

In [46]:
utils.init_distributed_mode(args)

device = torch.device(args.device)

# fix the seed for reproducibility
seed = args.seed + utils.get_rank()
torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

model, criterion, postprocessors = build_model_for_eval(args)
model.to(device)

dataset_test = build_dataset(image_set='val', args=args)

sampler_test = torch.utils.data.SequentialSampler(dataset_test)

data_loader_test = DataLoader(dataset_test, args.batch_size, sampler=sampler_test,
                             drop_last=False, collate_fn=utils.collate_fn, num_workers=args.num_workers)

base_ds = get_coco_api_from_dataset(dataset_test)

args.exit = 0

if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_test, base_ds, device, args)
    # if args.output_dir:
        # utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")


Not using distributed mode
loading annotations into memory...
Done (t=0.31s)
creating index...
index created!
Evaluation for exit at layer- 0
Test:  [  0/292]  eta: 0:15:51  class_error: 36.20  sub_error: 56.79  obj_error: 55.56  rel_error: 76.54  loss: 4.2945 (4.2945)  loss_ce: 0.6475 (0.6475)  loss_bbox: 1.2174 (1.2174)  loss_giou: 1.5004 (1.5004)  loss_rel: 0.9292 (0.9292)  loss_ce_unscaled: 0.6475 (0.6475)  class_error_unscaled: 36.1963 (36.1963)  sub_error_unscaled: 56.7901 (56.7901)  obj_error_unscaled: 55.5556 (55.5556)  loss_bbox_unscaled: 0.2435 (0.2435)  loss_giou_unscaled: 0.7502 (0.7502)  cardinality_error_unscaled: 6.0000 (6.0000)  loss_rel_unscaled: 0.9292 (0.9292)  rel_error_unscaled: 76.5432 (76.5432)  time: 3.2583  data: 1.2060  max mem: 5738
Test:  [100/292]  eta: 0:06:10  class_error: 40.30  sub_error: 50.79  obj_error: 42.86  rel_error: 63.49  loss: 3.9706 (3.9200)  loss_ce: 0.5455 (0.5512)  loss_bbox: 1.2845 (1.2501)  loss_giou: 1.3681 (1.3751)  loss_rel: 0.7496 (0

/usr/local/lib/python3.11/dist-packages/numpy/_core/fromnumeric.py:3596: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/usr/local/lib/python3.11/dist-packages/numpy/_core/_methods.py:138: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


Accumulating evaluation results...
DONE (t=10.47s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.075
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.172
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.057
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.011
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.039
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.121
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.141
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.224
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.229
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.041
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.156
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= l

In [47]:
args.exit = 1
if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_test, base_ds, device, args)
    # if args.output_dir:
        # utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")


Evaluation for exit at layer- 1
Test:  [  0/292]  eta: 0:14:41  class_error: 37.42  sub_error: 54.32  obj_error: 53.09  rel_error: 75.31  loss: 4.1239 (4.1239)  loss_ce: 0.6380 (0.6380)  loss_bbox: 1.1670 (1.1670)  loss_giou: 1.4368 (1.4368)  loss_rel: 0.8822 (0.8822)  loss_ce_unscaled: 0.6380 (0.6380)  class_error_unscaled: 37.4233 (37.4233)  sub_error_unscaled: 54.3210 (54.3210)  obj_error_unscaled: 53.0864 (53.0864)  loss_bbox_unscaled: 0.2334 (0.2334)  loss_giou_unscaled: 0.7184 (0.7184)  cardinality_error_unscaled: 6.6000 (6.6000)  loss_rel_unscaled: 0.8822 (0.8822)  rel_error_unscaled: 75.3086 (75.3086)  time: 3.0192  data: 1.2225  max mem: 6650
Test:  [100/292]  eta: 0:05:44  class_error: 35.07  sub_error: 49.21  obj_error: 47.62  rel_error: 65.08  loss: 3.8765 (3.7702)  loss_ce: 0.5450 (0.5321)  loss_bbox: 1.2129 (1.1956)  loss_giou: 1.3015 (1.3152)  loss_rel: 0.7482 (0.7272)  loss_ce_unscaled: 0.5450 (0.5321)  class_error_unscaled: 37.0968 (36.6561)  sub_error_unscaled: 56.000

In [51]:
args.exit = 2
if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_test, base_ds, device, args)
    # if args.output_dir:
        # utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")


Evaluation for exit at layer- 2
Test:  [  0/292]  eta: 0:20:32  class_error: 37.42  sub_error: 55.56  obj_error: 53.09  rel_error: 74.07  loss: 4.0038 (4.0038)  loss_ce: 0.6355 (0.6355)  loss_bbox: 1.1166 (1.1166)  loss_giou: 1.3811 (1.3811)  loss_rel: 0.8705 (0.8705)  loss_ce_unscaled: 0.6355 (0.6355)  class_error_unscaled: 37.4233 (37.4233)  sub_error_unscaled: 55.5556 (55.5556)  obj_error_unscaled: 53.0864 (53.0864)  loss_bbox_unscaled: 0.2233 (0.2233)  loss_giou_unscaled: 0.6905 (0.6905)  cardinality_error_unscaled: 7.1000 (7.1000)  loss_rel_unscaled: 0.8705 (0.8705)  rel_error_unscaled: 74.0741 (74.0741)  time: 4.2199  data: 1.9918  max mem: 7152
Test:  [100/292]  eta: 0:05:46  class_error: 35.82  sub_error: 49.21  obj_error: 46.03  rel_error: 65.08  loss: 3.7745 (3.6756)  loss_ce: 0.5471 (0.5182)  loss_bbox: 1.1663 (1.1633)  loss_giou: 1.2523 (1.2735)  loss_rel: 0.7521 (0.7206)  loss_ce_unscaled: 0.5471 (0.5182)  class_error_unscaled: 37.1429 (35.6319)  sub_error_unscaled: 54.430

In [52]:
args.exit = 3
if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_test, base_ds, device, args)
    # if args.output_dir:
        # utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")


Evaluation for exit at layer- 3
Test:  [  0/292]  eta: 0:18:39  class_error: 34.97  sub_error: 55.56  obj_error: 51.85  rel_error: 71.60  loss: 3.8871 (3.8871)  loss_ce: 0.6151 (0.6151)  loss_bbox: 1.0875 (1.0875)  loss_giou: 1.3447 (1.3447)  loss_rel: 0.8399 (0.8399)  loss_ce_unscaled: 0.6151 (0.6151)  class_error_unscaled: 34.9693 (34.9693)  sub_error_unscaled: 55.5556 (55.5556)  obj_error_unscaled: 51.8519 (51.8519)  loss_bbox_unscaled: 0.2175 (0.2175)  loss_giou_unscaled: 0.6723 (0.6723)  cardinality_error_unscaled: 6.1000 (6.1000)  loss_rel_unscaled: 0.8399 (0.8399)  rel_error_unscaled: 71.6049 (71.6049)  time: 3.8351  data: 1.7426  max mem: 7152
Test:  [100/292]  eta: 0:05:42  class_error: 37.31  sub_error: 46.03  obj_error: 46.03  rel_error: 66.67  loss: 3.6775 (3.5985)  loss_ce: 0.5399 (0.5048)  loss_bbox: 1.1700 (1.1428)  loss_giou: 1.2173 (1.2371)  loss_rel: 0.7218 (0.7137)  loss_ce_unscaled: 0.5399 (0.5048)  class_error_unscaled: 36.2069 (35.6960)  sub_error_unscaled: 53.164

In [53]:
args.exit = 4
if args.eval:
    test_stats, coco_evaluator = evaluate(model, criterion, postprocessors, data_loader_test, base_ds, device, args)
    # if args.output_dir:
        # utils.save_on_master(coco_evaluator.coco_eval["bbox"].eval, output_dir / "eval.pth")

Evaluation for exit at layer- 4
Test:  [  0/292]  eta: 0:17:09  class_error: 34.36  sub_error: 51.85  obj_error: 49.38  rel_error: 72.84  loss: 3.8459 (3.8459)  loss_ce: 0.6012 (0.6012)  loss_bbox: 1.0841 (1.0841)  loss_giou: 1.3342 (1.3342)  loss_rel: 0.8265 (0.8265)  loss_ce_unscaled: 0.6012 (0.6012)  class_error_unscaled: 34.3558 (34.3558)  sub_error_unscaled: 51.8519 (51.8519)  obj_error_unscaled: 49.3827 (49.3827)  loss_bbox_unscaled: 0.2168 (0.2168)  loss_giou_unscaled: 0.6671 (0.6671)  cardinality_error_unscaled: 6.2000 (6.2000)  loss_rel_unscaled: 0.8265 (0.8265)  rel_error_unscaled: 72.8395 (72.8395)  time: 3.5261  data: 1.3018  max mem: 7152
Test:  [100/292]  eta: 0:05:42  class_error: 40.30  sub_error: 47.62  obj_error: 47.62  rel_error: 68.25  loss: 3.6236 (3.5314)  loss_ce: 0.5379 (0.4968)  loss_bbox: 1.1374 (1.1237)  loss_giou: 1.1696 (1.2040)  loss_rel: 0.7341 (0.7070)  loss_ce_unscaled: 0.5379 (0.4968)  class_error_unscaled: 36.9369 (35.9108)  sub_error_unscaled: 51.898

In [54]:
from google.colab import runtime
runtime.unassign()